use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import random
import numpy as np
import pandas as pd
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle


In [2]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [38]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 2048
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id].to('cuda:2')
            product_emb = product_embeddings[batch_product_id].to('cuda:2')
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g  

# Merge valid score

In [6]:
FIELD_NAME = 'gru4rec_scores_2'

In [7]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [8]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [24]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_DE/product_embeddings_2023-06-01-20-52-24.pt'
DE_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_DE/valid_embeddings_2023-06-01-20-52-30.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_JP/product_embeddings_2023-06-01-20-53-24.pt'
JP_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_JP/valid_embeddings_2023-06-01-20-53-30.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_UK/product_embeddings_2023-06-01-20-54-25.pt'
UK_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_UK/valid_embeddings_2023-06-01-20-54-31.pt'

In [10]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/b5aeac4e5b9ff0518bbcb59a28086594'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/2536617955df215e0047f5b220d1c012'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/8d133ea55ad67bd3efd625dfeff0fb1d'

In [11]:
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [12]:
merged_candidates = read_merged_candidates_feature()
valid_sessions = read_valid_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [25]:
# sess embeddings 
valid_DE_query_emb = torch.load(DE_valid_embeddings_path, map_location='cpu')
valid_JP_query_emb = torch.load(JP_valid_embeddings_path, map_location='cpu')
valid_UK_query_emb = torch.load(UK_valid_embeddings_path, map_location='cpu')
valid_query_embeddings = torch.empty(len(valid_sessions), EMBED_DIM)
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'DE'].index).tolist()] = valid_DE_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'JP'].index).tolist()] = valid_JP_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'UK'].index).tolist()] = valid_UK_query_emb

In [26]:
# product_embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [27]:
DE_product_emb.shape, JP_product_emb.shape, UK_product_emb.shape

(torch.Size([518328, 128]),
 torch.Size([395010, 128]),
 torch.Size([500181, 128]))

In [28]:
merged_candidates_gru4rec = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [29]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [30]:
# merged_candidates_gru4rec_g = cudf.from_pandas(merged_candidates_gru4rec)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [31]:
# merged_candidates_gru4rec_score_g = merged_candidates_gru4rec_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_gru4rec_score_g['dataset_id'] = merged_candidates_gru4rec_score_g['dataset_id'].fillna(0)
# merged_candidates_gru4rec_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_gru4rec_score_g = merged_candidates_gru4rec_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_gru4rec_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_gru4rec_score = merged_candidates_gru4rec_score_g.to_pandas()

In [32]:
merged_candidates_gru4rec_score = merged_candidates_gru4rec.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_gru4rec_score['dataset_id'] = merged_candidates_gru4rec_score['dataset_id'].fillna(0)
merged_candidates_gru4rec_score.drop(columns=['locale'], inplace=True)
merged_candidates_gru4rec_score = merged_candidates_gru4rec_score.sort_values(by=['sess_id', 'product'])
merged_candidates_gru4rec_score.reset_index(drop=True, inplace=True)
# merged_candidates_gru4rec_score = merged_candidates_gru4rec_score_g.to_pandas()

In [33]:
# del merged_candidates_gru4rec_g
# del product_id_df_g
# del merged_candidates_gru4rec_score_g

In [34]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_gru4rec_score['dataset_id'][merged_candidates_gru4rec_score['sess_locale'] == locale] = \
        merged_candidates_gru4rec_score['dataset_id'][merged_candidates_gru4rec_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_118421/3067966839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_gru4rec_score['dataset_id'][merged_candidates_gru4rec_score['sess_locale'] == locale] = \


In [35]:
merged_candidates_gru4rec_score['dataset_id'].max()

1413518.0

In [36]:
product_embeddings.shape

torch.Size([1413519, 128])

In [39]:
merged_candidates_gru4rec_score[FIELD_NAME] = get_scores(merged_candidates_gru4rec_score, valid_query_embeddings, product_embeddings)

  0%|          | 0/38498 [00:00<?, ?it/s]/tmp/ipykernel_118421/1566316832.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|██████████| 38498/38498 [05:55<00:00, 108.34it/s]


In [40]:
normalize_scores(merged_candidates_gru4rec_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [41]:
merged_candidates[FIELD_NAME] = merged_candidates_gru4rec_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_gru4rec_score['normalized_'+FIELD_NAME]

In [32]:
cast_dtype(merged_candidates)
merged_candidates.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [42]:
merged_candidates

,sess_id,sess_locale,product,target,sess_avg_price,product_price,product_freq,sasrec_scores_3,normalized_sasrec_scores_3,sasrec_scores_2,normalized_sasrec_scores_2,seqmlp_scores,normalized_seqmlp_scores,narm_scores,normalized_narm_scores,gru4rec_scores_2,normalized_gru4rec_scores_2
0,0,DE,355165591X,0.0,43.256542,8.990000,51.0,2.230508,7.658405e-09,0.512931,1.377575e-09,6.044256,3.054954e-09,3.628220,1.469988e-13,5.093824,1.288155e-10
1,0,DE,3833237058,0.0,43.256542,22.000000,84.0,9.605231,1.221631e-05,9.325538,9.255110e-06,10.732503,3.319589e-07,15.734776,2.661486e-08,13.133082,3.993682e-07
2,0,DE,B00CIXSI6U,0.0,43.256542,6.470000,7.0,0.714114,1.681035e-09,-0.115904,7.345399e-10,4.902086,9.749146e-10,7.462822,6.802375e-12,4.295491,5.797709e-11
3,0,DE,B00NVDOWUW,0.0,43.256542,11.990000,166.0,8.750996,5.199363e-06,8.507557,4.084482e-06,13.401752,4.789881e-06,9.774860,6.866981e-11,10.362890,2.502041e-08
4,0,DE,B00NVDP3ZU,0.0,43.256542,22.990000,502.0,8.056712,2.596729e-06,5.898870,3.007453e-07,9.162767,6.908073e-08,10.392188,1.273116e-10,12.687778,2.558471e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,0.0,9.383333,16.990000,7.0,6.813615,1.076201e-03,7.203015,4.597607e-04,6.808200,4.075167e-04,7.742151,7.126370e-04,7.978776,9.559332e-04
78842195,261815,UK,B0BCX6QB4L,0.0,9.383333,10.990000,53.0,9.030836,9.881445e-03,10.123234,8.526421e-03,10.454871,1.562696e-02,10.198983,8.314902e-03,10.432762,1.112193e-02
78842196,261815,UK,B0BFPJYXQL,0.0,9.383333,10.560000,7.0,0.796892,2.623396e-06,1.711608,1.895152e-06,-0.960521,1.722791e-07,2.838342,5.286535e-06,2.107661,2.695469e-06
78842197,261815,UK,B0BH3X67S3,0.0,9.383333,6.830000,38.0,4.250781,8.296004e-05,6.447586,2.159998e-04,5.091795,7.323526e-05,6.170062,1.479513e-04,7.305784,4.876977e-04


In [43]:
# verify gru4rec scores
merged_candidates[merged_candidates['sess_id'] == 150001].sort_values(by=['gru4rec_scores_2'], ascending=False)[['sess_locale', 'product', 'normalized_gru4rec_scores_2', 'gru4rec_scores_2', 'normalized_sasrec_scores_2', 'sasrec_scores_2']].iloc[:15]

,sess_locale,product,normalized_gru4rec_scores_2,gru4rec_scores_2,normalized_sasrec_scores_2,sasrec_scores_2
45142726,JP,B01G4IIOAO,0.348916,17.139351,0.235870,17.185946
45142762,JP,B075FTW3PD,0.286218,16.941273,0.146815,16.711842
45142729,JP,B01HM6XIVM,0.163295,16.380081,0.107268,16.397999
45142698,JP,B00JX88YJE,0.065610,15.468241,0.311983,17.465614
45142699,JP,B00JX88YJY,0.047178,15.138441,0.106716,16.392834
45142791,JP,B07G93RR4S,0.029798,14.678961,0.021031,14.768680
45142702,JP,B00LSCAO1E,0.026865,14.575335,0.016529,14.527805
45142728,JP,B01GR9UTLM,0.008740,13.452435,0.017104,14.561987
45142680,JP,B009GYL0LO,0.005261,12.944850,0.002587,12.673185
45142663,JP,B000WIDOL2,0.005137,12.920975,0.003376,12.939500
